In [ ]:
import pandas as pd
import numpy as np
import torch
import os
import gc


In [ ]:
import pickle

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [ ]:
import cv2

In [ ]:

import torchvision.models.detection as tmd
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor as FRP
import inspect
import torchvision
#converting the customised rcnn model
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor


In [ ]:
root_dir = 'D:\\Drone-Object-Detection\\VisDroneDataset\\'
changed_dir = 'D:\\Drone-Object-Detection\\VisDroneDataset_changed\\'
test_dir = 'test\\'
train_dir = 'train\\'
val_dir = 'validation\\'
labels_map = ['Background', 'Pedestrian', 'People', 'Bycicle', 'Car', 'Van', 'Truck',
                'Tricycle', 'Awning-tricycle', 'Bus', 'Motor', 'Others']

In [ ]:
def normalize_hist(channel_array):
    def Sh(h,i):
        return sum(h[:i])
    h = np.bincount(channel_array.flatten())
    H = sum(h)
    h = h/H
    
    for i in range(channel_array.shape[0]):
        for j in range(channel_array.shape[1]):
            channel_array[i][j] = 255 * Sh(h,channel_array[i][j])
                                         
    return channel_array

In [ ]:
def color_histogram_equalization(image):
    
    b,g,r = image[0].astype(np.uint8),image[1].astype(np.uint8),image[2].astype(np.uint8)
    
    b_eq = normalize_hist(b)
    g_eq = normalize_hist(g)
    r_eq = normalize_hist(r)
    
    equalized_image = cv2.merge((b_eq, g_eq, r_eq))
    #equalized_image = np.stack((b_eq, g_eq, r_eq))
    return equalized_image



In [ ]:

def hsv_histogram_equalization(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv_image)
    v_eq = cv2.equalizeHist(v)
    equalized_hsv = cv2.merge((h, s, v_eq))
    equalized_image = cv2.cvtColor(equalized_hsv, cv2.COLOR_HSV2BGR)
    
    return equalized_image

In [ ]:
def no_croppping(to_save,idx, image_path,bounding_boxes):
    to_file=''
    for index, box in enumerate(bounding_boxes): 
        x1_b, y1_b,width , height, a, b, c,d = box
        to_file+=f"{x1_b},{y1_b},{width},{height},{a},{b},{c},{d}\n"


    opencvImage = cv2.imread(image_path)
    cv2.imwrite(f'{to_save}train\\smart_images\\output_image_not_changed_{idx}.png',opencvImage)
    with open(f'{to_save}train\\smart_annotations\\bounding_boxes_not_changed_{idx}.txt', 'w') as bbox_file:
                bbox_file.write(to_file)

In [ ]:
from PIL import Image

def crop_and_save_boxes(to_save, image_path, txt_path,idx,img_shape=(480,480)):
    # Load the image
    original_image = Image.open(image_path)   
    bounding_boxes=[]
    with open(txt_path, 'r') as file:#check whether 0.8 are small
        bounding_boxes = np.array([list(map(int, line.split(','))) for line in file.readlines()])
        
        
        
    
    w_new = img_shape[0]
    h_new = img_shape[1]
    w,h = original_image.size
    # Calculate the number of 480x480 boxes to create
    num_boxes_horizontal = w// w_new  # Ceiling division
    num_boxes_vertical = h // h_new

    # Iterate over boxes and crop the image
    for i in range(num_boxes_horizontal):
        for j in range(num_boxes_vertical):
            # Calculate box coordinates
            x1_new = i * w_new
            y1_new =  j * h_new
            x2_new = (i+1) * w_new
            y2_new = (j+1) * h_new
            

            # Crop the image
            cropped_image = original_image.crop((x1_new, y1_new, x2_new, y2_new))

            # Save the cropped image
            #cropped_image.save(f'{to_save}train\\images_cropped\\output_image_{idx}_{i}_{j}.png')
            opencvImage = cv2.cvtColor(np.array(cropped_image), cv2.COLOR_RGB2BGR)
            
            to_file=''
            bb_new = []
            obs_num=0
            needed_el = 0
            for index, box in enumerate(bounding_boxes):
                
                x1_b, y1_b,width , height, a, b, c,d = box
                x2_b = x1_b + width
                y2_b = y1_b + height


                if (x1_b > x2_new or y1_b > y2_new) or (x2_b < x1_new or y2_b < y1_new): # no itersection

                    continue
                else:
                    
                    x1_b_new = max(0, x1_b - x1_new)
                    y1_b_new = max(0, y1_b - y1_new)
                    x2_b_new = min(x2_new - x1_new, x2_b - x1_new)
                    y2_b_new = min(y2_new - y1_new, y2_b - y1_new)#bug here
                    
                    to_file+=f"{x1_b_new},{y1_b_new},{x2_b_new - x1_b_new},{y2_b_new - y1_b_new},{a},{b},{c},{d}\n"
                    obs_num+=1
                    #needed_el += (b==6 or b==7 or b==8 or b==9)
                    if(x1_b_new < 0 or y1_b_new < 0 or  x2_b_new > 480 or y2_b_new > 480):
                        print(to_file,txt_path)
                        
                    
                #labels = bb_new[:,5]
                #small = [i  for i in bb_new if ]


            if(obs_num < 10) :
                continue

            with open(f'{to_save}train\\smart_annotations\\bounding_boxes_{idx}_{i}_{j}.txt', 'w') as bbox_file:
                bbox_file.write(to_file)
            cv2.imwrite(f'{to_save}train\\smart_images\\output_image_{idx}_{i}_{j}.png', opencvImage)


            

# Example usage
image_path = 'path/to/your/image.jpg'
txt_path = 'path/to/your/bounding_boxes.txt'


In [ ]:
def prepare_images(from_directory,to_directory, shape):
    images = os.listdir(from_directory+'images\\')
    annotations = os.listdir(from_directory+'annotations\\')
   
    for idx,(img_dir,ann_dir) in enumerate(zip(images, annotations)):
        
        image = cv2.imread(from_directory+'images\\'+ img_dir)  
        
        #image = sharp_img(image)
        
        img_width = image.shape[1]
        img_height = image.shape[0]
        
        new_width = shape[0]
        new_height = shape[1]
        if shape == (-1,-1):
            image = cv2.resize(image,(shape[0],shape[1]),interpolation = cv2.INTER_AREA)
        image = hsv_histogram_equalization(image)
        boxes = []
        labels = []
        with open(from_directory+'annotations\\'+ann_dir,'r') as f_f,open(to_directory+'annotations\\'+ann_dir,'w') as f_t:
            new_f = ''
            for line in f_f:
                boxes = []
                data = line.split(',')#strip
                #print(list(map(float, data[:5])))
                box_left, box_top, box_width, box_height,smth1, category,smth2,smth3 = map(float, data)
               
                box_width = 1 if box_width < 1 else box_width
                box_height = 1 if box_height < 1 else box_height
                
                box_right = box_left + box_width
                
                box_bottom = box_top + box_height
                #rescale box            
                box_left = (box_left/img_width) * new_width
                box_right = (box_right/img_width) * new_width
                box_top = (box_top/img_height) * new_height
                box_bottom = (box_bottom/img_height) * new_height
                boxes += [box_left,box_top,box_right, box_bottom]
                #print(boxes)
                new_f += ''.join([str(int(c))+',' for c in boxes])
                new_f+=str(int(smth1))+','
                new_f += str(int(category))+','
                new_f += str(int(smth2))+','
                new_f += str(int(smth3))+'\n'
                
                
            f_t.write(new_f)
            
        
        if(idx % 100==0):
            print(f'imf N {idx} written')
        #print('img written')
        cv2.imwrite(to_directory+'images\\'+ img_dir,image)
        

In [ ]:
#prepare_images(root_dir+train_dir,changed_dir+train_dir ,(480,480))
def crop():
    images = os.listdir(root_dir+train_dir+'images\\')
    annotations = os.listdir(root_dir+train_dir+'annotations\\')

    for idx,(img_dir,ann_dir) in enumerate(zip(images, annotations)):
        crop_and_save_boxes(changed_dir, root_dir+train_dir+'images\\'+img_dir,root_dir+train_dir+'annotations\\'+ann_dir,\
                            idx,img_shape = (480,480))
        if(idx % 100 == 0):
            print(idx)

In [ ]:
crop()

In [ ]:
class VisDroneDataset(Dataset):
    def __init__(self, path, img_shape):
        
        self.root = path
        self.width = img_shape[1]
        self.height = img_shape[0]
        self.images = os.listdir(path+'images\\')
        self.annotations = os.listdir(path+'annotations\\')
        self.length = len(self.images)
        
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        #print(self.root+'images\\'+self.images[idx])
        image = cv2.imread(self.root+'images\\'+self.images[idx])
        #print(image.shape)
        img_width = image.shape[1]
        img_height = image.shape[0]     #right?
        
        image  =cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        if ((self.width,self.height) != (-1,-1)):
            image = cv2.resize(image,(self.width,self.height),interpolation = cv2.INTER_AREA)
        else:
            self.width = img_width
            self.height = img_height
            
        image = image.transpose(2,0,1)
        image = (image/255.0)
        
        
        #bbox_left, bbox_top, bbox_width, bbox_height, category = map(float, data[:5])
        target = self.annotations[idx]    #*.txt file       
        
        boxes = []
        labels = []
        with open(self.root+'annotations\\'+target) as f:
            for line in f:
                data = line.split(',')#strip
                #print(list(map(float, data[:5])))
                box_left, box_top, box_width, box_height,smth, category = map(float, data[:6])
                
                box_width = 1 if box_width < 1 else box_width
                box_height = 1 if box_height < 1 else box_height
                
                box_right = box_left + box_width
                box_bottom = box_top + box_height
                #rescale box            
                box_left = (box_left/img_width) * self.width
                box_right = (box_right/img_width) * self.width
                box_top = (box_top/img_height) * self.height
                box_bottom = (box_bottom/img_height) * self.height
                boxes.append((box_left,box_top,box_right, box_bottom))
#                 for b in boxes:
#                     for bb in b:
#                         if bb > 480 or bb < 0:
#                             print(b,'\n',bb)
                labels.append(int(category))
                
        
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        #print(boxes)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor((idx))
        
        return image, target # np.array, dict of lists

In [ ]:
train_dataset = VisDroneDataset(changed_dir+train_dir,(480,480))
val_dataset = VisDroneDataset(root_dir+val_dir,(-1,-1))

In [ ]:
def collate_fn(batch):
    images = []
    targets = []

    for img, target in batch:
        images.append(img)
        targets.append(target)

    return (images, targets)


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True,collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=3, shuffle=True,collate_fn=collate_fn)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
torch.hub.set_dir('D:\\torch_models')

In [ ]:
from torchvision.models import resnet50

In [ ]:
import torchvision
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights
from torchvision.models.detection import ssdlite320_mobilenet_v3_large

In [ ]:
def create_model_ssd():
    model = ssdlite320_mobilenet_v3_large(num_classes=12)
    print(model)
    return model

In [ ]:

def create_model_faster_rcnn(num_classes):
    # Load a pre-trained  R-CNN model 
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
    for p in model.parameters():
        p.requires_grad=False
        
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    return model



In [ ]:
def load_best_model():
    with open('model.pkl','rb') as f:
        model = pickle.load(f)
        return  model
    

In [ ]:
model = create_model_faster_rcnn(12)

In [ ]:
model.train()


In [ ]:
import time
from tqdm import tqdm
def train_one_epoch(model, train_loader, optimizer):
    total_loss=0
    n=0
    prev_time = time.time()
    for data, target in tqdm(train_loader):
        
        optimizer.zero_grad()
        #print(len(data))
        data = torch.as_tensor(np.array(data),dtype=torch.float32).to(device)
        target = [{k: v.to(device) for k, v in t.items()} for t in target]
        
        #print(target)
        loss_dict = model(data, target)

        losses = sum(loss for loss in loss_dict.values())

        total_loss+=losses.item()
        losses.backward()
        optimizer.step()
        if(n % 100 == 0):
            print('loss', total_loss/(n+1))
            prev_time = time.time()
            
        
        n+=1
        del data, target
        #model = model.to(torch.device('cpu'))
        gc.collect()
        torch.cuda.empty_cache()
        
        #model = model.to(device)
    
    print(total_loss/n,'a')

if(n % 1000 == 0):
            model.eval()
            with torch.no_grad():
                preds = model(data)
                
            nms_preds=[]
            
            for pred in preds:
                nms_pred = non_max_suppression_git(pred)
                nms_preds.append(nms_pred)
                
            map_val = calc_mAP(nms_pred,annotations)['map'].item()
            print(map_val)
            model.train()
            

In [ ]:
num_epoch = 50
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.0005, weight_decay=0.000005)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

model = model.to(device)


print(len(params))
print(len(list(model.parameters())))

In [ ]:

for epoch in range(num_epoch):
    train_one_epoch(model,train_loader,optimizer)
    lr_scheduler.step()

In [ ]:
#with open('model_trained_on_cropped.pkl', 'wb') as f:
#    pickle.dump(model, f)

In [ ]:
#with open('model_trained_on_cropped.pkl', 'rb') as f:
#    model = pickle.load(f)

In [ ]:
model = load_best_model().to(device)

In [ ]:
model.eval()

In [ ]:

gc.collect()

torch.cuda.empty_cache()

In [ ]:
#!pip install -U git+https://github.com/obss/sahi

In [ ]:
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction, predict, get_prediction
from sahi.utils.file import download_from_url
from sahi.utils.cv import read_image
#from IPython.display import Image
from PIL import Image

In [ ]:
def box_iou_batch(boxes_a: np.ndarray, boxes_b: np.ndarray) -> np.ndarray:

    def box_area(box):
        return (box[2] - box[0]) * (box[3] - box[1])

    area_a = box_area(boxes_a.T)
    area_b = box_area(boxes_b.T)

    top_left = np.maximum(boxes_a[:, None, :2], boxes_b[:, :2])
    bottom_right = np.minimum(boxes_a[:, None, 2:], boxes_b[:, 2:])

    area_inter = np.prod(
    	np.clip(bottom_right - top_left, a_min=0, a_max=None), 2)
        
    return area_inter / (area_a[:, None] + area_b - area_inter)

In [ ]:
def non_max_suppression(predictions: dict, iou_threshold: float = 0.9) -> np.ndarray:
    #print(predictions)
    predictions = torch.cat((predictions['boxes'] , predictions['scores'].unsqueeze(dim=1),\
                             predictions['labels'].unsqueeze(dim=1)),dim=1) .detach().cpu().numpy()
    
    
    sort_index = np.flip(predictions[:, 4].argsort())
    
    rows,columns = predictions.shape
    
    predictions = predictions[sort_index]

    boxes = predictions[:, :4]
    categories = predictions[:, 5]
    ious = box_iou_batch(boxes, boxes)
    ious = ious - np.eye(rows)

    keep = np.ones(rows, dtype=bool)

    for index, (iou, category) in enumerate(zip(ious, categories)):
        if not keep[index]:
            continue

        condition = (iou > iou_threshold) & (categories == category)
        keep = keep & ~condition

    return predictions[keep[sort_index.argsort()]]

In [ ]:
model.eval()


In [ ]:

import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
labels_map = ['Background', 'Pedestrian', 'People', 'Bycicle', 'Car', 'Van', 'Truck',
                'Tricycle', 'Awning-tricycle', 'Bus', 'Motor', 'Others']
def draw_bounding_boxes(img: np.ndarray, boxes: np.ndarray,labels: np.ndarray,scores: np.ndarray, ax):   # image - tensor, others - numpy
    
    ax.imshow(img.detach().cpu().numpy().transpose(1,2,0))
    ax.set_title('title')
    threshold = 0.2
    for box,label,score in zip(boxes,labels.astype(np.uint8),scores):
        #print(label)
        if(score < threshold):
            continue
        x1, y1, x2, y2 = box
        width = x2 - x1
        height = y2 - y1
        rect = patches.Rectangle((x1, y1), width, height, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        ax.text(
                x1,
                y1,
                f"{labels_map[label]}: {(np.round(score*100,1))}%",
                verticalalignment='top',
                color='white',
                fontsize=8,
                bbox={'facecolor': 'red', 'alpha': 0.7, 'pad': 1}
            )
    #plt.show()
    return ax

In [ ]:
from torchvision.ops import nms

def non_max_suppression_git(preds: dict, iou_threshold=0.5, score_threshold=0.2):
    
    boxes = preds['boxes']
    scores = preds['scores']
    labels = preds['labels']

    # Filter out predictions below the score_threshold
    keep_idx = scores > score_threshold
    boxes = boxes[keep_idx]
    scores = scores[keep_idx]
    labels = labels[keep_idx]

    # Perform NMS and get the indices of the remaining predictions
    keep_indices = nms(boxes, scores, iou_threshold)

    # Filter out the predictions using the keep_indices
    nms_boxes = boxes[keep_indices]
    nms_scores = scores[keep_indices]
    nms_labels = labels[keep_indices]

    # Create a dictionary containing the filtered predictions
    nms_preds = {
        'boxes': nms_boxes,
        'scores': nms_scores,
        'labels': nms_labels
    }

    return nms_preds


In [ ]:
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [ ]:
def draw_predictions(imgs: list, preds: list, mode: str, i):#preds - list of dictionaries of tensors
    fig, ax = plt.subplots(1,len(imgs), figsize=(40,40))
    for idx,(img,ann) in enumerate(zip(imgs, preds)):
        #print(ann)
        annotations_new = []
        annotations_new.append(torch.cat((ann['boxes'] ,\
                                           (ann['scores']).unsqueeze(dim=1) if mode=='pred' else torch.ones(len(ann['boxes'])).\
                                          unsqueeze(dim=1),\
                                           ann['labels'].unsqueeze(dim=1)),dim=1).detach().cpu().numpy())
        ann = np.array(annotations_new[0])
        #print(ann)
        draw_bounding_boxes(img,ann[:,0:4],ann[:,5],ann[:,4],ax[idx] if len(imgs) != 1 else ax)
        
    if(i!=-1):
        fig.savefig(f'D:\\Drone-Object-Detection\\Results\\Images\\img{i}.png')
        
    plt.show()

In [ ]:
from torchmetrics.detection import MeanAveragePrecision

In [ ]:
def replace_cuda_with_cpu(data):
    if isinstance(data, torch.Tensor):
        return data.cpu()
    elif isinstance(data, dict):
        return {key: replace_cuda_with_cpu(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [replace_cuda_with_cpu(item) for item in data]
    else:
        return data

In [ ]:
def dict_to_cpu(preds):
    new_l = []
    for dic in preds:
        new_l.append(replace_cuda_with_cpu(dic))
        
    return new_l

In [ ]:
def from_dict_to_arr(dict):
    annotations_new = []
    for ann in dict:
        
        annotations_new+=(torch.cat((ann['boxes'] ,\
                                           torch.ones(len(ann['boxes'])).unsqueeze(dim=1).to(device),\
                                           ann['labels'].unsqueeze(dim=1)),dim=1).detach().cpu().numpy())
        
    return annotations_new

In [ ]:
map_file = open('Results\\mAPs\\file.txt','w')
map_file.close()



In [ ]:
def calc_mAP(preds: list,target: list):
    metric = MeanAveragePrecision(iou_type="bbox")
    metric.update(preds, annotations)
    #map_file.write(str(metric.compute())+'\n')
    #print(metric.compute())
    return (metric.compute())


In [ ]:
map_file = open('Results\\mAPs\\file.txt','a')
#path to img - 'Results\\imgs\\img(i).jpg

In [ ]:
#model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)

In [ ]:
detection_model = AutoDetectionModel.from_pretrained(
    model_type='torchvision',
    model=model,
    confidence_threshold=0.5,
    image_size=1080,
    device="cuda:0", 
    load_at_init=True,
)

In [ ]:
def from_x1y1wh_to_x1y1x2y2(box: list)->list:
    
    box[2] = box[0] + box[2]
    box[3] = box[1] + box[3]
    return box

In [ ]:
def to_common_dict(list_of_dict):
    #print(list_of_dict)
    boxes = []
    labels = []
    scores = []
    for dic in list_of_dict:
        #print(dic['bbox'])
        boxes.append(from_x1y1wh_to_x1y1x2y2(dic['bbox']))
        labels.append(dic['category_id'])##not sure
        scores.append(dic['score'])
        
    return {'scores':torch.tensor(scores),'labels':torch.tensor(labels), 'boxes':torch.tensor(boxes)}

In [ ]:
def predict_with_SAHI(batch_imgs, detection_model):
    results = []
    for img in batch_imgs:
        result = get_sliced_prediction(
            Image.fromarray(np.moveaxis((img * 255).astype(np.uint8), [0,1,2],[2,0,1])),#2 1 0
            detection_model,
            slice_height = 480,
            slice_width = 480,
            overlap_height_ratio = 0.2,
            overlap_width_ratio = 0.2 
        ) # returns x1y1wh
        results.append(to_common_dict(result.to_coco_annotations()))
        
    return results

In [ ]:
def predict_without_SAHI(batch_imgs, model):
    batch_imgs = [torch.from_numpy(img).float().to(device) for img in batch_imgs]
    with torch.no_grad():
        preds = model(batch_imgs)#tensor
    return preds

In [ ]:
val_loader_for_map = DataLoader(val_dataset, batch_size=1, shuffle=True,collate_fn=collate_fn)

In [ ]:
maps = ""
c = 0
total_map_sahi = 0
total_map_no_sahi = 0
results=[]
for i, (batch_imgs, annotations) in enumerate(val_loader_for_map):
    
    results_sahi = predict_with_SAHI(batch_imgs, detection_model)
    results_no_sahi = predict_without_SAHI(batch_imgs,model)
    
    preds_sahi = replace_cuda_with_cpu(results_sahi) 
    preds_no_sahi = replace_cuda_with_cpu(results_no_sahi) 
    
    nms_preds_sahi = []
    for pred_sahi in preds_sahi:
        nms_pred_sahi = non_max_suppression_git(pred_sahi)       
        nms_preds_sahi.append(nms_pred_sahi)
        
    
    nms_preds_no_sahi = []
    for pred_no_sahi in preds_no_sahi:
        nms_pred_no_sahi = non_max_suppression_git(pred_no_sahi)       
        nms_preds_no_sahi.append(nms_pred_no_sahi)
    #batch_imgs = [torch.from_numpy(img).float().to(device) for img in batch_imgs]#tensor
    #draw_predictions(batch_imgs,nms_preds,'pred',1)#list of dictionaries of tensors
    #draw_predictions(batch_imgs,annotations,'tar',-1)

    #print(nms_preds)
    total_map_sahi +=calc_mAP(nms_preds_sahi,annotations)['map'].item()
    total_map_no_sahi +=calc_mAP(nms_preds_no_sahi,annotations)['map'].item()
    c+=1
    
    if(i == 5):
        break
    if(i % 20 == 0):
        print(f'{i}, With SAHI map: {total_map_sahi / c}')
        print(f'{i}, Without SAHI map: {total_map_no_sahi / c}')
        
print(f'With SAHI map: {total_map_sahi / c}')
print(f'Without SAHI map: {total_map_no_sahi / c}')
#Trouble SAHI with dataset
#D:\Drone-Object-Detection\\VisDroneDataset\\validation\\images\\0000001_04527_d_0000008.jpg

In [ ]:
map_file.close()

In [ ]:
#BETTER PEDESTRIANS WORSE CARS

In [ ]:
print([1,2,3] in [1,2,3,4])

In [ ]:
result = predict_with_SAHI([1], detection_model)